# Example-Based Model

## Initialization of the Datasets


### Source Dataset

#### Storing the Tagalog Part of Speech Data Set

In [ ]:
import pandas as pd

# Read the tagalog POS dataset
tl_pos_data = pd.read_json('src/json data/tl_pos.json')

tl_pos_data.head()

#### Storing the Ilokano Part of Speech Data Set

In [ ]:
# Read the ilokano POS dataset
il_pos_data = pd.read_json('src/json data/il_pos.json')

il_pos_data.head()

#### Storing the Tagalog Part of Speech Structure

In [ ]:
dict_sen_poss = pd.DataFrame(tl_pos_data['POS'])

dict_sen_poss.columns = ['Tagalog POS']

#### Storing the Ilokano Part of Speech 

In [ ]:
dict_sen_poss['Ilokano POS'] = il_pos_data['POS']

dict_sen_poss.head(50)

### Verb Tagalog to Ilokano Dictionary

In [ ]:
tl_sen_poss_list = dict_sen_poss['Tagalog POS']
il_sen_poss_list = dict_sen_poss['Ilokano POS']
"""
putting the POS of the sentences in a list object
"""

dict_tl_il_vb = pd.DataFrame(columns=['Tagalog Verb', 'Ilokano Verb'])
dict_tl_il_nn = pd.DataFrame(columns=['Tagalog Noun', 'Ilokano Noun'])

## Appending in the List

### For Verb List

In [ ]:
def append_vb_list(tl_verb, tl_verb_list, il_verb, il_verb_list, curr_il_pos, next_il_pos, next2_il_pos, matched, sp_index, wp_index):
    if tl_verb not in tl_verb_list:
        """
        if the verb is not in the list
        """
        tl_verb_list.append(tl_verb)
        inDict = False
        
    else:
        """
        if the verb is in the list
        """
        temp_index = tl_verb_list.index(tl_verb)
        inDict = True

    """
    append the the verb in the tagalog verb
    """

    if curr_il_pos == 'VB' and wp_index not in matched:
        """
        if VB : VB
        if the Ilokano POS is a verb
        """
        temp_verb = il_pos_data['Tokenized'][sp_index][wp_index]
        il_verb.append(temp_verb)
        matched.append(wp_index)
        
    elif curr_il_pos == 'DT' and next_il_pos == 'VB' and wp_index not in matched:    
        """
        if VB : DT VB
        if the Ilokano POS is a determiner and the next POS is a verb
        eg. Nilalang : ti Aramid
        """
        temp_curr_verb = il_pos_data['Tokenized'][sp_index][wp_index]
        temp_next_verb = il_pos_data['Tokenized'][sp_index][wp_index + 1]
        temp_verb = temp_curr_verb + ' ' + temp_next_verb
        il_verb.append(temp_verb)
        matched.append(wp_index)
        matched.append(wp_index + 1) 

    elif curr_il_pos == 'NN' and next_il_pos == 'VB' and wp_index not in matched:    
        """
        if VB : NN VB
        if the Ilokano POS is a determiner and the next POS is a verb
        """
        temp_verb = il_pos_data['Tokenized'][sp_index][wp_index + 1]
        il_verb.append(temp_verb)
        matched.append(wp_index + 1)
        
    elif curr_il_pos == 'DT' and next_il_pos == 'NN' and next2_il_pos == 'VB' and wp_index not in matched:
        """
        if VB DT NN : DT NN VB
        if the Ilokano POS is a determiner and the next POS is a verb
        """
        temp_verb = il_pos_data['Tokenized'][sp_index][wp_index + 2]
        il_verb.append(temp_verb)
        matched.append(wp_index + 2)
        
    else:
        """
        if VB : Other POS
        if the Ilokano POS is not a verb
        """
        il_verb.append('None')
        matched.append(wp_index)

    if not inDict:
        il_verb_list.append(il_verb)
    else:
        if il_verb[0] not in il_verb_list[temp_index]:
            il_verb_list[temp_index].append(il_verb[0])
            

### For Noun List

In [ ]:
def append_nn_list(tl_noun, tl_noun_list, il_noun, il_noun_list, curr_il_pos, next_il_pos, matched, sp_index, wp_index):
    if tl_noun not in tl_noun_list:
        """
        if the noun is not in the list
        """
        tl_noun_list.append(tl_noun)
        inDict = False
        
    else:
        """
        if the noun is in the list
        """
        temp_index = tl_noun_list.index(tl_noun)
        inDict = True

    """
    append the the noun in the tagalog noun
    """

    if curr_il_pos == 'NN':
        """
        if NN : NN
        if the Ilokano POS is a noun
        """
        temp_noun = il_pos_data['Tokenized'][sp_index][wp_index]
        il_noun.append(temp_noun)
        matched.append(wp_index)
        
        
    elif curr_il_pos == 'DT' and next_il_pos == 'NN':    
        """
        if NN : DT NN
        if the Ilokano POS is a determiner and the next POS is a noun
        """
        temp_curr_noun = il_pos_data['Tokenized'][sp_index][wp_index]
        temp_next_noun = il_pos_data['Tokenized'][sp_index][wp_index + 1]
        temp_noun = temp_curr_noun + ' ' + temp_next_noun
        il_noun.append(temp_noun) 
        matched.append(wp_index)
          
    else:
        """
        if NN : Other POS
        if the Ilokano POS is not a noun
        """
        il_noun.append('None')
        matched.append(wp_index)

    if not inDict:
        il_noun_list.append(il_noun)
    else:
        if il_noun[0] not in il_noun_list[temp_index]:
            il_noun_list[temp_index].append(il_noun[0])

### Tagalog to Ilokano Matcher

In [ ]:

wp_index = None # word position index

"""
instantiating the verb lists
"""

def match_tl_il_pos():
    """
    This function matches the POS of the sentences in the Tagalog and Ilokano datasets
    """
    tl_verb_list = []
    il_verb_list = []
    tl_noun_list = []
    il_noun_list = []
    tl_to_il_verb_list = []
    sp_index = 0
    """
    instantiating the verb lists
    """
    
    for tl_sen_pos in tl_sen_poss_list:
        # loop for getting the pos structure of every sentence
        """
        tl_sen is a list of POS of a sentence
        eg. ['VB', 'DT', 'NN', 'DT', 'NN']
        """
        matched = []
        il_sen = il_sen_poss_list[sp_index]
    
        wp_index = 0
        """
        instantiating the variables
        """
        
        for tl_word_pos in tl_sen_pos:
            # loop for each pos in a sentence
            """
            tl_word_pos is a POS of a word
            eg. 'VB'
            """
            il_verb = []
            il_noun = []
            
            tl_word = tl_pos_data['Tokenized'][sp_index][wp_index]
            # gets the word in every sentence
            
            try:
                curr_il_pos = il_sen[wp_index] # ti
            except IndexError:
                curr_il_pos = 'None'
            try:
                next_il_pos = il_sen[wp_index + 1]
            except IndexError:
                next_il_pos = 'None'
            try:
                next2_il_pos = il_sen[wp_index + 2]
            except IndexError:
                next2_il_pos = 'None'
            try:
                prev_il_pos = il_sen[wp_index - 1]
                if (wp_index - 1) < 0:
                    prev_il_pos = 'None'
            except IndexError:
                prev_il_pos = 'None'
            """
            getting the current, next, and previous POS in the sentence
            """
            
            # Matching Conditions
            # 1. VB
            if tl_word_pos == 'VB':
                """
                Verb Matching
                if the POS is a verb, append the index of the verb to the verb list
                """
                append_vb_list(tl_word, tl_verb_list, il_verb, il_verb_list, curr_il_pos, next_il_pos, next2_il_pos, matched, sp_index, wp_index)

            if tl_word_pos == 'NN':
                """
                Noun Matching
                if the POS is a noun, append the index of the noun to the noun list
                """
                append_nn_list(tl_word, tl_noun_list, il_noun, il_noun_list, curr_il_pos, next_il_pos, matched, sp_index, wp_index)
                            
            wp_index += 1     
        sp_index += 1
    
    dict_tl_il_vb['Tagalog Verb'] = tl_verb_list
    dict_tl_il_vb['Ilokano Verb'] = il_verb_list
    dict_tl_il_nn['Tagalog Noun'] = tl_noun_list
    dict_tl_il_nn['Ilokano Noun'] = il_noun_list
    
    
match_tl_il_pos()
dict_tl_il_vb.head(7)
dict_tl_il_nn.head(30)

In [ ]:
il_pos_data['Tokenized'].head(7)


In [ ]:
dict_tl_il_vb['Ilokano Verb'][3]